In [ ]:
import logging
import math,random
import time

import torch
import torch.nn as nn
import torch.nn.functional as F

from models.Attention import Attention
from models.Embeddings.BERT import BERT_Embedding
from models.Embeddings.GLOVE import GLOVE_Embedding
from models.Encoders.CNN import CNN_Encoder
from models.Encoders.RNN import RNN_User_Encoder
from models.Interactors.CNN import CNN_Interactor
from models.Interactors.FIM import FIM_Interactor
from models.Interactors.KNRM import KNRM_Interactor
from models.Interactors.BERT_Overlook import BERT_Interactor
from models.Interactors.BERT_Onepass import BERT_Interactor

from models.ESM import ESM

from data.configs.demo import config
from utils.utils import prepare
from utils.Manager import Manager

from models.Modules.DRM import DRM_Matching
# from models.Modules.TFM import TFM

In [ ]:
config.his_size = 10
config.embedding = 'bert'
config.hidden_dim = 768
config.device = 1
config.learning_rate = 1e-6

# config.path = "C:/"
config.mode = 'dev'

manager = Manager(config)

vocab, loaders = prepare(manager)
record = next(iter(loaders[0]))

In [ ]:
embedding = BERT_Embedding(manager)
encoderN = CNN_Encoder(manager)
encoderU = RNN_User_Encoder(manager)
docReducer = DRM_Matching(manager)
# termFuser = TFM(manager.his_size, manager.k)
# interactor = CNN_Interactor(manager)

interactor = BERT_Interactor(manager)

esm = ESM(manager, embedding, encoderN, encoderU, docReducer, None, interactor).to(manager.device)

In [ ]:
record

In [ ]:
esm.eval()

In [5]:
import torch

In [6]:
a = torch.zeros(2,2,3)
b = torch.rand(1,3)

a[0,:] = b

In [7]:
a

tensor([[[0.9778, 0.7345, 0.6286],
         [0.9778, 0.7345, 0.6286]],

        [[0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000]]])

In [2]:
from transformers import BertForPreTraining

In [3]:
a = BertForPreTraining.from_pretrained('bert-base-uncased')

Some weights of BertForPreTraining were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['cls.predictions.decoder.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
a

BertForPreTraining(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine